In [2]:
import html5lib
import requests
import lxml
from bs4 import BeautifulSoup
from bs4 import Comment
import pandas as pd
import numpy as np

In [69]:
# Get the roster table from the placeholder comment
def is_comment(element): 
    return isinstance(element, Comment)
url = 'https://www.baseball-reference.com/teams/ARI/2019.shtml'
html = requests.get(url).content
soup =  BeautifulSoup(html, 'html5lib')
appe = soup.find(attrs={'id':'all_appearances'}).find(text=is_comment)
roster = pd.read_html(str(BeautifulSoup(appe).find(attrs={'id':'appearances'})))[0]

#Removing duplicated headers
roster = roster.drop(roster[roster['Age'] == 'Age'].index)

#Renaming the columns
newColsNames = {'Name':'name', 'Age':'age','Unnamed: 2':'country','B':'batting_side','T':'throwing_hand','Ht':'height',
                'Wt':'weight', 'DoB':'date_of_birth','Yrs':'years','Salary':'salary', 'Batting':'batting'}
roster.rename(newColsNames, axis=1, inplace=True)
roster = roster[['name', 'age', 'country', 'batting_side', 'height', 'weight', 'years', 'batting','salary']]

#Country is duplicated on the same column. Getting only one instance.
roster['country'] = roster['country'].str.split(expand=True)[1]

#Converting height into total inches.
roster['height'] = roster['height'].str.split(expand=True)[0].str.replace("'",'').astype(int) * 12 + roster['height'].str.split(expand=True)[1].str.replace('"','').astype(int)
roster['weight'] = pd.to_numeric(roster['weight'])
roster['salary'] = roster['salary'].str.replace('$','').str.replace(',','').astype(float)
# There are some duplicated rows, I removed them:
roster.drop_duplicates(inplace=True)

In [71]:
roster.head()

,name,age,country,batting_side,height,weight,years,batting,salary
0,Nick Ahmed,29,US,R,74,200,6,158,3663000.0
1,Abraham Almonte,30,DO,B,70,223,7,17,NaN
2,Matt Andriese,29,US,R,74,215,5,51,920000.0
3,Alex Avila,32,US,L,71,210,11,63,4250000.0
4,Archie Bradley,26,US,R,76,215,5,61,1830000.0
